Environment Setup and Dataset Download

In [ ]:
!pip install --upgrade tensorflow_federated

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"mewkhi","key":"51a23160bc635731f18e0b9a7bc75b53"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d hereisburak/pins-face-recognition
!mkdir PinsData
!unzip -q pins-face-recognition.zip -d ./PinsData

Applying Noise on Raw Dataset to Generate Perturbed Dataset

In [ ]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import cv2
import matplotlib.pyplot as plt

# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the path to the dataset directory
dataset_path = "/content/PinsData/105_classes_pins_dataset"

# Define the transform for the DataLoader used for visualization (with normalization)
transform_visualize = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the ImageFolder dataset with normalization
visualize_dataset = ImageFolder(dataset_path, transform=transform_visualize)

# Define the network architecture
class AlexNet(nn.Module):
    def __init__(self, num_classes=105):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

learning_rate = 0.004

# Initialize the global model
global_model = AlexNet().to(device)

# Define the optimizer and loss function for the global model
optimizer = optim.Adam(global_model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

# Load the dataset
dataset = ImageFolder(dataset_path, transform=transform_visualize)

# Create a directory to save the perturbed images
perturbed_dir = "perturbed_images"
os.makedirs(perturbed_dir, exist_ok=True)

# Define the epsilon value for the DeepFool attack
epsilon = 0.03

# Modified DeepFool attack to increase perturbation
def deepfool_attack_batch(images, model, epsilon, reduction_factor=0.95):
    model.eval()
    perturbed_images = images.clone().detach().requires_grad_(True).to(device)

    with torch.no_grad():
        outputs = model(images)
        original_labels = torch.argmax(outputs, dim=1)

    outputs_grad = None
    while True:
        outputs = model(perturbed_images)
        predicted_labels = torch.argmax(outputs, dim=1)

        if torch.all(predicted_labels == original_labels):
            break

        if perturbed_images.grad is not None:
            perturbed_images.grad.data.zero_()

        loss = criterion(outputs, original_labels)
        loss.backward(retain_graph=True)

        if perturbed_images.grad is not None and outputs_grad is None:
            outputs_grad = perturbed_images.grad.data.clone()

        if outputs_grad is not None:
            perturbation = torch.abs(loss) / torch.norm(outputs_grad.view(images.size(0), -1), dim=1).view(-1, 1, 1, 1)
            perturbed_images = perturbed_images + perturbation * outputs_grad / torch.norm(outputs_grad.view(images.size(0), -1), dim=1).view(-1, 1, 1, 1)
            perturbed_images = torch.clamp(perturbed_images, 0, 1)
            perturbed_images = perturbed_images.clone().detach().requires_grad_(True).to(device)
        else:
            raise ValueError("No gradient computed for perturbed_images. The attack can't proceed.")

    return perturbed_images.detach()

# Apply the modified DeepFool attack to the entire dataset and save the perturbed images
batch_size = 64
total_images = len(visualize_dataset)
num_batches = (total_images + batch_size - 1) // batch_size

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, total_images)

    # Get a batch of images and labels
    images, labels = zip(*[visualize_dataset[i] for i in range(start_idx, end_idx)])
    images = torch.stack(images).to(device)
    labels = torch.tensor(labels).to(device)

    # DeepFool attack for the batch of images
    perturbed_images = deepfool_attack_batch(images, global_model, epsilon)

    for j in range(end_idx - start_idx):
        # Convert the perturbed image to a numpy array and change the channel order
        perturbed_image_np = perturbed_images[j].squeeze().detach().cpu().numpy().transpose(1, 2, 0)

        # Convert the image to the BGR color format for saving
        perturbed_image_bgr = cv2.cvtColor((perturbed_image_np * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

        # Get the class label of the image
        class_label = visualize_dataset.classes[labels[j]]

        # Create a subdirectory for the class label if it does not exist
        class_subdir = os.path.join(perturbed_dir, class_label)
        os.makedirs(class_subdir, exist_ok=True)

        # Save the perturbed image in the class subdirectory
        perturbed_image_path = os.path.join(class_subdir, f"perturbed_{start_idx + j}.jpg")
        cv2.imwrite(perturbed_image_path, perturbed_image_bgr)

        # Undo normalization on the original image
        original_image_np = images[j].squeeze().detach().cpu().numpy().transpose(1, 2, 0)
        original_image_np = (original_image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])) * 255
        original_image_np = original_image_np.astype(np.uint8)

        # Convert the image to the BGR color format for visualization
        original_image_bgr = cv2.cvtColor(original_image_np, cv2.COLOR_RGB2BGR)

Raw Dataset in Normal Machine Learning Environment

In [ ]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Specify the path to the dataset directory
dataset_path = "/home/ec2-user/SageMaker/PinsData/105_classes_pins_dataset"

# Define the network architecture with Batch Normalization
class AlexNetWithBN(nn.Module):
    def __init__(self, num_classes=105):
        super(AlexNetWithBN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(192),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(256),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

# Initialize the global model with Batch Normalization
global_model = AlexNetWithBN().to(device)

# Define the optimizer and loss function for the global model with weight decay
learning_rate = 0.0001
optimizer = optim.Adam(global_model.parameters(), lr=learning_rate, weight_decay=1e-4)
criterion = nn.NLLLoss()

# Load the dataset
dataset = ImageFolder(dataset_path, transform=transform)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader objects for training and testing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Training function with learning rate scheduling
def train_model(model, criterion, optimizer, train_loader, device, epoch):
    model.train()
    running_loss = 0.0
    correct_predictions = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / len(train_loader.dataset) * 100
    return epoch_loss, epoch_accuracy

# Testing function
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_accuracy = correct_predictions / len(test_loader.dataset) * 100
    return epoch_loss, epoch_accuracy

# Training and testing loop with learning rate scheduling
num_epochs = 50
train_losses, train_accuracies, test_losses, test_accuracies = [], [], [], []

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(global_model, criterion, optimizer, train_loader, device, epoch)
    test_loss, test_accuracy = test_model(global_model, criterion, test_loader, device)
    scheduler.step()

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print("-" * 30)

# Plot the training and testing accuracy and loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label='Train')
plt.plot(range(1, num_epochs+1), test_losses, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Testing Loss')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), train_accuracies, label='Train')
plt.plot(range(1, num_epochs+1), test_accuracies, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Testing Accuracy')

plt.tight_layout()
plt.show()

Perturbed Dataset in Normal Machine Learning Environment

In [ ]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import torchvision.models as models

# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the path to the dataset directory
dataset_path = "/home/ec2-user/SageMaker/perturbed_images/"

batch_size = 64

# Reduced data augmentation for the training set
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No data augmentation for the testing set
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define datasets with proper transformations
train_dataset = ImageFolder(dataset_path, transform=train_transform)
test_dataset = ImageFolder(dataset_path, transform=test_transform)

# Initialize the model with VGG16 and Dropout
class VGG16WithDropout(nn.Module):
    def __init__(self, num_classes=105):
        super(VGG16WithDropout, self).__init__()
        vgg16 = models.vgg16(pretrained=True)
        self.features = vgg16.features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

# Initialize the model with VGG16 and Dropout
global_model = VGG16WithDropout().to(device)

# Define the optimizer and loss function for the global model with weight decay
learning_rate = 0.0001
weight_decay = 1e-5
optimizer = optim.Adam(global_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.NLLLoss()

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Training function with learning rate scheduling and early stopping
def train_model(model, criterion, optimizer, train_loader, device, epoch):
    model.train()
    running_loss = 0.0
    correct_predictions = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / len(train_loader.dataset) * 100
    return epoch_loss, epoch_accuracy

# Testing function
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_accuracy = correct_predictions / len(test_loader.dataset) * 100
    return epoch_loss, epoch_accuracy

# Training and testing loop with learning rate scheduling and early stopping
num_epochs = 30
train_losses, train_accuracies, test_losses, test_accuracies = [], [], [], []

best_test_accuracy = 0.0
patience = 15
early_stopping_counter = 0

# Use ReduceLROnPlateau scheduler instead of StepLR
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=5, verbose=True)

best_model_wts = None

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(global_model, criterion, optimizer, train_loader, device, epoch)
    test_loss, test_accuracy = test_model(global_model, criterion, test_loader, device)

    # Use the scheduler
    scheduler.step(test_accuracy)

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print("-" * 30)

    # Save the best model weights
    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        best_model_wts = global_model.state_dict()
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping...")
            break

global_model.load_state_dict(best_model_wts)

# Plot the training and testing accuracy and loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label='Train')
plt.plot(range(1, num_epochs+1), test_losses, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Testing Loss')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), train_accuracies, label='Train')
plt.plot(range(1, num_epochs+1), test_accuracies, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Testing Accuracy')

plt.tight_layout()
plt.show()